In [1]:
from PIL import Image
import os
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np
import pandas as pd
from IPython.display import display, Markdown

import sys
sys.path.append('../../rse-panimg')

from pathlib import Path
from panimg import convert

In [2]:
ATTENTION_GIF_DIR_DIR = f"V:/experiments/lung-malignancy-fairness-shaurya/nlst/sybil_attentions/attention"
ATTENTION_PNG_DIR_DIR = f"V:/experiments/lung-malignancy-fairness-shaurya/nlst/sybil_attentions/attention_imgs"
ATTENTION_MHA_DIR_DIR = f"V:/experiments/lung-malignancy-fairness-shaurya/nlst/sybil_attentions/attention_mhas"

In [3]:
os.makedirs(ATTENTION_MHA_DIR_DIR, exist_ok=True)

In [3]:
subfolders = os.listdir(ATTENTION_PNG_DIR_DIR)
subfolders

['serie_1.2.840.113654.2.55.145385352624582326596824382275465435553',
 'serie_1.2.840.113654.2.55.145616115778906978668823205981054792515',
 'serie_1.2.840.113654.2.55.158391584353757200962479457670501546291',
 'serie_1.2.840.113654.2.55.175702525661871626890784804368160596565',
 'serie_1.2.840.113654.2.55.241924049353992159785903857377772231263',
 'serie_1.2.840.113654.2.55.248083914288744344546769925024268560749',
 'serie_1.2.840.113654.2.55.263132066860645282583134020193623337530',
 'serie_1.2.840.113654.2.55.287137212573085949544120272517078094143',
 'serie_1.2.840.113654.2.55.55905915368746417680608580988846234578',
 'serie_1.2.840.113654.2.55.67095101227495808685302586770473894957',
 'serie_1.2.840.113654.2.55.81136962133262551156371928517766508555',
 'serie_1.3.6.1.4.1.14519.5.2.1.7009.9004.117168587881036721196334662566',
 'serie_1.3.6.1.4.1.14519.5.2.1.7009.9004.145215505102406333449483782713',
 'serie_1.3.6.1.4.1.14519.5.2.1.7009.9004.181715871926844141208177392303',
 'serie_

In [25]:
[s[6:] for s in subfolders]

['1.2.840.113654.2.55.145385352624582326596824382275465435553',
 '1.2.840.113654.2.55.145616115778906978668823205981054792515',
 '1.2.840.113654.2.55.158391584353757200962479457670501546291',
 '1.2.840.113654.2.55.175702525661871626890784804368160596565',
 '1.2.840.113654.2.55.241924049353992159785903857377772231263',
 '1.2.840.113654.2.55.248083914288744344546769925024268560749',
 '1.2.840.113654.2.55.263132066860645282583134020193623337530',
 '1.2.840.113654.2.55.287137212573085949544120272517078094143',
 '1.2.840.113654.2.55.55905915368746417680608580988846234578',
 '1.2.840.113654.2.55.67095101227495808685302586770473894957',
 '1.2.840.113654.2.55.81136962133262551156371928517766508555',
 '1.3.6.1.4.1.14519.5.2.1.7009.9004.117168587881036721196334662566',
 '1.3.6.1.4.1.14519.5.2.1.7009.9004.145215505102406333449483782713',
 '1.3.6.1.4.1.14519.5.2.1.7009.9004.181715871926844141208177392303',
 '1.3.6.1.4.1.14519.5.2.1.7009.9004.184145865626087655808498226297',
 '1.3.6.1.4.1.14519.5.2

In [ ]:
for i in range(1):
    seriesid = subfolders[i]
    n_frames = len(os.listdir(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"))
    print(f"{seriesid}: {n_frames} frames")
    result = convert(
        input_directory=Path(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"),
        output_directory=Path(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"),
    )
    print(result)

serie_1.2.840.113654.2.55.145616115778906978668823205981054792515: 171 frames


KeyboardInterrupt: 

In [4]:
import SimpleITK as sitk
ATTENTION_MHA3D_DIR = f"V:/experiments/lung-malignancy-fairness-shaurya/nlst/sybil_attentions/attention_mha3d"
os.makedirs(ATTENTION_MHA3D_DIR, exist_ok=True)

def stack_mha_files(input_folder, output_mha_path):
    # Get list of MHA files in the folder, sorted by filename (assuming sequential naming)
    frame_subdirs = sorted(os.listdir(input_folder))
    mha_files = [f'{sd}/{os.listdir(f"{input_folder}/{sd}")[0]}' for sd in frame_subdirs][1:]
    
    # mha_files = sorted([f for f in os.listdir(input_folder) if f.endswith('.mha')])

    if not mha_files:
        raise ValueError("No MHA files found in the provided directory")

    # Read the first MHA file to determine the size and spacing of the slices
    first_mha_path = os.path.join(input_folder, mha_files[0])
    first_image = sitk.ReadImage(first_mha_path, outputPixelType=sitk.sitkVectorUInt8)
    slice_size = first_image.GetSize()  # (width, height, depth)
    print("slice size:", slice_size, ", num_components:", first_image.GetNumberOfComponentsPerPixel())

    # Create an empty list to hold the images
    image_list = []

    # Load each MHA file and append the images to the list
    for i, mha_file in enumerate(mha_files):
        mha_path = os.path.join(input_folder, mha_file)
        img = sitk.ReadImage(mha_path, sitk.sitkVectorUInt8)
        # print("slice ", i, " size:", img.GetSize(), ", num_components:", img.GetNumberOfComponentsPerPixel())
        if img.GetNumberOfComponentsPerPixel() == 3:
            image_list.append(img)

    # Stack the images along the third dimension (Z-axis)
    print("number of frames in stack:", len(image_list))
    stacked_image = sitk.JoinSeries(image_list)
    stack_size = stacked_image.GetSize()
    print("stack size:", stack_size, ", stack n_components:", stacked_image.GetNumberOfComponentsPerPixel())

    # Save the stacked image as a single 3D MHA file
    sitk.WriteImage(stacked_image, output_mha_path)
    print(f"Stacked MHA saved as {output_mha_path}")


In [19]:
# Example usage
ATTENTION_MHA3D_DIR = f"V:/experiments/lung-malignancy-fairness-shaurya/nlst/sybil_attentions/attention_mha3d_v2"
os.makedirs(ATTENTION_MHA3D_DIR, exist_ok=True)

for i in range(1):
    seriesid = subfolders[i]
    n_frames = len(os.listdir(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"))
    print(f"{seriesid}: {n_frames} subfolders")
    input_folder = f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"  # Folder containing the individual MHA slices
    output_mha_path = f"{ATTENTION_MHA3D_DIR}/{seriesid}.mha"  # Path to save the stacked MHA file
    stack_mha_files(input_folder, output_mha_path)

serie_1.2.840.113654.2.55.145385352624582326596824382275465435553: 136 subfolders
slice size: (512, 512) , num_components: 3
slice  0  size: (512, 512) , num_components: 3
slice  1  size: (512, 512) , num_components: 3
slice  2  size: (512, 512) , num_components: 3
slice  3  size: (512, 512) , num_components: 3
slice  4  size: (512, 512) , num_components: 3
slice  5  size: (512, 512) , num_components: 3
slice  6  size: (512, 512) , num_components: 3
slice  7  size: (512, 512) , num_components: 3
slice  8  size: (512, 512) , num_components: 3
slice  9  size: (512, 512) , num_components: 3
slice  10  size: (512, 512) , num_components: 3
slice  11  size: (512, 512) , num_components: 3
slice  12  size: (512, 512) , num_components: 3
slice  13  size: (512, 512) , num_components: 3
slice  14  size: (512, 512) , num_components: 3
slice  15  size: (512, 512) , num_components: 3
slice  16  size: (512, 512) , num_components: 3
slice  17  size: (512, 512) , num_components: 3
slice  18  size: (512

In [22]:
ATTENTION_MHA3D_DIR = f"V:/experiments/lung-malignancy-fairness-shaurya/nlst/sybil_attentions/attention_mha3d_v2"
os.makedirs(ATTENTION_MHA3D_DIR, exist_ok=True)

for i in range(24, len(subfolders)):
    seriesid = subfolders[i]
    # n_frames = len(os.listdir(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"))
    # print(f"{seriesid}: {n_frames} frames")
    # result = convert(
    #     input_directory=Path(f"{ATTENTION_PNG_DIR_DIR}/{seriesid}"),
    #     output_directory=Path(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"),
    # )
    # print(result)

    n_frames = len(os.listdir(f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"))
    print(f"{seriesid}: {n_frames} subfolders")
    input_folder = f"{ATTENTION_MHA_DIR_DIR}/{seriesid}"  # Folder containing the individual MHA slices
    output_mha_path = f"{ATTENTION_MHA3D_DIR}/{seriesid}.mha"  # Path to save the stacked MHA file
    stack_mha_files(input_folder, output_mha_path)

serie_1.3.6.1.4.1.14519.5.2.1.7009.9004.652542111531875496786085581001: 204 subfolders
slice size: (512, 512) , num_components: 3
number of frames in stack: 203
stack size: (512, 512, 203) , stack n_components: 3
Stacked MHA saved as V:/experiments/lung-malignancy-fairness-shaurya/nlst/sybil_attentions/attention_mha3d_v2/serie_1.3.6.1.4.1.14519.5.2.1.7009.9004.652542111531875496786085581001.mha
